In [4]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import logging
logging.basicConfig(level=logging.INFO)
from tqdm import tqdm
import numpy as np
import pandas as pd


ImportError: cannot import name 'is_tokenizers_available' from 'transformers.utils' (/usr/local/miniconda3/envs/tsum/lib/python3.8/site-packages/transformers/utils/__init__.py)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
data_sents= pd.read_csv("../data/Test Data/subtaskA_test_data.csv", index_col="id")
data_ans = pd.read_csv("../data/Test Data/subtaskA_gold_answers.csv", header=None)
num_total = len(data_sents)
data_test = [(data_ans.iloc[i, 1], tuple(data_sents.iloc[i, :])) for i in range(num_total)]

In [37]:
def score(model, tokenizer, sentence):
    """
    Compute the loss of the sentence output by the MLM model.
    """
    mask_token_id = tokenizer.mask_token_id
    non_mask_token_id = -100 
    tensor_input = tokenizer(sentence, return_tensors='pt')["input_ids"]
    repeat_input = tensor_input.repeat(tensor_input.size(-1)-2, 1)
    mask = torch.ones(tensor_input.size(-1) - 1).diag(1)[:-2]
    masked_input = repeat_input.masked_fill(mask == 1, mask_token_id)
    labels = repeat_input.masked_fill(masked_input != mask_token_id, non_mask_token_id)
    masked_input = masked_input.to(device)
    labels = labels.to(device)
    with torch.no_grad():
        loss = model(masked_input, labels=labels)["loss"]
    return loss.item()

In [54]:
# Load tokenizer and model parameters
# Training process is complicated and directly using command line runing run_mlm_no_trainer.py,
# which is not recorded.
# Directly using untrained models: 
# model_path = "../pretrained_models/bert-base-uncased"
# tokenizer = BertTokenizer.from_pretrained(model_path)
# model = BertForMaskedLM.from_pretrained(model_path).to(device)


tokenizer = BertTokenizer.from_pretrained('./BertMLM-finetuned')
model = BertForMaskedLM.from_pretrained('./BertMLM-finetuned').to(device)

In [55]:
# test
num_correct = 0

for label, sentences in tqdm(data_test):
    ppls = np.array([score(model, tokenizer, sentence) for sentence in sentences])
    answer = np.argmax(ppls)
    if answer == label:
        num_correct += 1

print(f"correct/total: {num_correct}/{num_total}")
print(f"Accuracy: {num_correct / num_total}")   

 10%|████████████▍                                                                                                           | 104/1000 [00:03<00:27, 32.74it/s]

correct/current: 75/100


 20%|████████████████████████▌                                                                                               | 205/1000 [00:05<00:24, 32.61it/s]

correct/current: 148/200


 30%|████████████████████████████████████▌                                                                                   | 305/1000 [00:08<00:21, 32.59it/s]

...

correct/current: 603/800


 91%|████████████████████████████████████████████████████████████████████████████████████████████████████████████▊           | 907/1000 [00:25<00:02, 34.53it/s]

correct/current: 676/900


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:28<00:00, 35.13it/s]

correct/current: 751/1000
correct/total: 751/1000
Accuracy: 0.751
